In [ ]:
from openai import OpenAI
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
from pymongo import MongoClient
from pymongo.mongo_client import MongoClient
import os

#API KEY
# client = OpenAI()

In [ ]:
CUNET_EMAIL = "student.chula.ac.th"
CUNET_PASSWORD = ""

In [ ]:
driver = webdriver.Chrome()
driver.get('https://www.scopus.com/')

driver.implicitly_wait(3)

sign_up_button = driver.find_element(By.ID, value="signin_link_move")
sign_up_button.click()

email_box = driver.find_element(By.ID, value="bdd-email")
email_box.send_keys(CUNET_EMAIL)

driver.implicitly_wait(3)

cookie_button = driver.find_element(By.ID, value="onetrust-accept-btn-handler")
cookie_button.click()

driver.implicitly_wait(3)

continue_button = driver.find_element(By.ID, value="bdd-elsPrimaryBtn")
continue_button.click()

driver.implicitly_wait(3)


password_box = driver.find_element(By.ID, value="bdd-password")
password_box.send_keys(CUNET_PASSWORD)

driver.implicitly_wait(3)

sign_in_button = driver.find_element(By.ID, value="bdd-elsPrimaryBtn")
sign_in_button.click()

driver.implicitly_wait(20)

In [ ]:
pub_date_scrape_list = pd.read_csv('Pub_Date_2017_Scrape_List.csv')['Eid'].to_list()

In [ ]:
len(pub_date_scrape_list)

In [ ]:
month_year_list = []
error_list = []
index = 0

for eid in pub_date_scrape_list:
    try:
        url = 'https://www.scopus.com/record/display.uri?eid=' + eid + '&origin=resultslist&sort=plf-f&src=s&sid=4070e132d327bfd204e04a25e45084e4&sot=a&sdt=a&s=AFFILORG(Chulalongkorn+University)+AND+PUBYEAR+%3D+2017&sl=53&sessionSearchId=4070e132d327bfd204e04a25e45084e4&relpos=3'

        driver.get(url)

        element = driver.find_element(By.CSS_SELECTOR, "[class^='PublicationInformationBar-module']")
        send_text = element.text

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "I will give you string of text of the article detail you must exact the last publication month and year from the string and return it in format MM-YYYY if it don't have date or month just return 00 for missing field and JUST RETURN ONLY THE DATE STRING"},
                {
                    "role": "user",
                    "content": send_text
                }
            ]
        )
        month_year = completion.choices[0].message.content
        month = month_year.split('-')[0]
        year = month_year.split('-')[1]
        print(index, eid, month, year)
        month_year_list.append([eid, month, year])
        index += 1
    except:
        error_list.append(eid)


In [ ]:
month_year_df = pd.DataFrame(month_year_list, columns=['Eid', 'Public_Month', 'Public_Year'])

In [ ]:
month_year_df.to_csv('Scrape_Data_Pub_Date.csv',index=False)

In [ ]:
print(error_list)